This notebook will compare model architecture, hyperparameters, inputs, and training/validation error

In [1]:
from src.crossval import run_crossval
import xarray as xr

In [2]:
level = "small"
input_dir = "data/processed_data"
target_dir = "data/processed_data"
stats_dir = "data/processed_data"

In [3]:
inputs = xr.open_zarr(f"{input_dir}/train_inputs_{level}.zarr")
targets = xr.open_dataset(f"{target_dir}/train_targets.nc")
stats = xr.open_dataset(f"{stats_dir}/daily_input_stats_{level}.nc")

In [4]:
inputs

<xarray.Dataset>
Dimensions:                       (latitude: 16, longitude: 21, day: 332,
                                   tod: 4, level: 5)
Coordinates:
  * day                           (day) datetime64[ns] 2002-04-07 ... 2019-05-20
  * latitude                      (latitude) float32 45.0 44.0 ... 31.0 30.0
  * level                         (level) int64 925 850 700 500 300
  * longitude                     (longitude) float32 255.0 256.0 ... 275.0
  * tod                           (tod) int64 0 6 12 18
Data variables: (12/13)
    10m_u_component_of_wind       (latitude, longitude, day, tod) float32 dask.array<chunksize=(16, 21, 332, 4), meta=np.ndarray>
    10m_v_component_of_wind       (latitude, longitude, day, tod) float32 dask.array<chunksize=(16, 21, 332, 4), meta=np.ndarray>
    2m_dewpoint_temperature       (latitude, longitude, day, tod) float32 dask.array<chunksize=(16, 21, 332, 4), meta=np.ndarray>
    2m_temperature                (latitude, longitude, day, tod) float32 dask.array<chunksize=(16, 21, 332, 4), meta=np.ndarray>
    geopotential                  (level, latitude, longitude, day, tod) float32 dask.array<chunksize=(5, 16, 21, 332, 4), meta=np.ndarray>
    geopotential_at_surface       (latitude, longitude, day, tod) float32 dask.array<chunksize=(16, 21, 332, 4), meta=np.ndarray>
    ...                            ...
    specific_humidity             (level, latitude, longitude, day, tod) float32 dask.array<chunksize=(5, 16, 21, 332, 4), meta=np.ndarray>
    temperature                   (level, latitude, longitude, day, tod) float32 dask.array<chunksize=(5, 16, 21, 332, 4), meta=np.ndarray>
    toa_incident_solar_radiation  (latitude, longitude, day, tod) float32 dask.array<chunksize=(16, 21, 332, 4), meta=np.ndarray>
    u_component_of_wind           (level, latitude, longitude, day, tod) float32 dask.array<chunksize=(5, 16, 21, 332, 4), meta=np.ndarray>
    v_component_of_wind           (level, latitude, longitude, day, tod) float32 dask.array<chunksize=(5, 16, 21, 332, 4), meta=np.ndarray>
    vertical_velocity             (level, latitude, longitude, day, tod) float32 dask.array<chunksize=(5, 16, 21, 332, 4), meta=np.ndarray>

In [ ]:
# TODO: compare all models, save training and validation curves along with corresponding model name, data level, model/training hyperparameters

model_names = ["linear_regression"]
for name in model_names:
    scores = run_crossval(X, y, name, input_dim=X.shape[1], num_classes=3)
    print(f"{name}: {np.mean(scores):.3f} ± {np.std(scores):.3f}")